# Financial Entity Resolution

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Finance/7.EntityResolution.ipynb)

**Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Saving latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json to latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json


In [ ]:
from johnsnowlabs import * 
# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect
jsl.install()

👌 Detected license file /content/latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up John Snow Labs home in /home/ckl/.johnsnowlabs this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
Running "/usr/bin/python3 -m pip install https://pypi.johnsnowlabs.com/[LIBRARY_SECRET]spark-ocr/spark_ocr-4.0.1-py3-none-any.whl --force-reinstall"
Running "/usr/bin/python3 -m pip install https://pypi.johnsnowlabs.com/[LIBRARY_SECRET]spark-nlp-internal/spark_nlp_internal-4.1.0-py3-none-any.whl --force-reinstall"
Installing spark-nlp to /usr/bin/python3
Installing pyspark to /usr/bin/python3
Installed 3 products:
🕶 Spark-OCR installed! ✅ Empower your NLP with a set of eyes 
💊 Spark-Healthcare installed! ✅ Heal the planet with NLP! 
🚀 Spark-NLP installed! ✅ State of the art NLP at scale 
🍥 If you are on Google Colab, please restart your Notebook for cha

# Start Spark Session

In [ ]:
from johnsnowlabs import * 
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

👌 Detected license file /content/latest_3_1_x_spark_nlp_for_healthcare_spark_ocr_5112.json
📋 Stored new John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_2_for_Spark-Healthcare_Spark-OCR.json
👌 Launched SparkSession with Jars for: 🚀Spark-NLP, 💊Spark-Healthcare, 🕶Spark-OCR


In [5]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# if you want to start the session with custom params as in start function above
def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")
      
    return builder.getOrCreate()

#spark = start(SECRET)

# Sentence Entity Resolver Models

An NLP use case in financial or legal applications is identifying financial entities' presence in a given text. One of those entities could be `Company Name`. We can carry out NER to extract different chunks of information, but in real financial and legal use cases, the company name is usually not useful as it is mentioned in the text. Sometimes we need the _official_ name of the company (instead of `Amazon`, `Amazon.com INC`, as registered in Edgar). We have pre-trained sentence entity resolver models for these purposes shown below with the examples.

Other than providing the code in the "result" field it provides more metadata about the matching process:

- target_text -> Text to resolve
- resolved_text -> Best match text
- confidence -> Relative confidence for the top match (distance to probability)
- confidence_ratio -> Relative confidence for the top match. TopMatchConfidence / SecondMatchConfidence
- alternative_codes -> List of other plausible codes (in the KNN neighborhood)
- all_k_resolutions -> All codes descriptions
- all_k_results -> All resolved codes for metrics calculation purposes
- sentence -> SentenceId

### Helper Function
We will use following Generic Function For Getting the Codes and Relation Pairs

In [6]:
# returns LP resolution results

# import pandas as pd
pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='company_name', hcc=False):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []
    all_k_aux_labels=[]

    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))
        all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
        if hcc:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
        else:
            all_k_aux_labels.append([])

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes, 'all_codes':all_codes, 
                       'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})
    
    return df

## Sentence Entity Resolver (EDGAR)
[Edgar's Public Database](https://www.sec.gov/edgar/searchedgar/companysearch)

![image.png](attachment:635a0e2c-5d63-4a2b-be1a-f84aaf49e190.png)

- EDGAR, the Electronic Data Gathering, Analysis, and Retrieval system, is the primary system for companies and others submitting documents under the Securities Act of 1933, the Securities Exchange Act of 1934, the Trust Indenture Act of 1939, and the Investment Company Act of 1940. 

- Access to EDGAR’s public database is free—allowing you to research, for example, a public company’s financial information and operations by reviewing the filings the company makes with the SEC.(U.S. Securities and Exchange Commission)

Here we will normalize company names and find IRS code of each company with the Edgar's Database

### Company Name Normalization

`Company Name Normalization` is the process of obtaining the name of the company used by data providers, usually the "official" name of the company.

Sometimes, some data providers may have different versions of the name with different punctuation. For example, for Meta:
- Meta Platforms, Inc.
- Meta Platforms Inc.
- Meta Platforms, Inc
- etc

So, it's mandatory we do `Company Normalization` taking into account the database / datasource provider we want to extract data from. The data providers we have are:
- SEC Edgar
- Crunchbase until 2015
- Wikidata (in progress)

In [7]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("ner_chunk") \
      .setOutputCol("sentence_embeddings")
    
resolver = finance.SentenceEntityResolverModel.pretrained("finel_edgar_company_name", "en", "finance/models")\
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("normalized_name")\
      .setDistanceFunction("EUCLIDEAN")


pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          embeddings,
          resolver])

lp = LightPipeline(pipelineModel)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
finel_edgar_company_name download started this may take some time.
[OK!]


In [8]:
text = 'Stock Dividend Fund INC '

%time get_codes (lp, text, vocab='normalized_name')

CPU times: user 35.1 ms, sys: 3.19 ms, total: 38.3 ms
Wall time: 1.26 s


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,Stock Dividend Fund INC,0,23,Stock Dividend Fund Inc,"[Stock Dividend Fund Inc, FMI COMMON STOCK FUND INC, GENERAL MONEY MARKET FUND INC, NATIONAL INVESTORS CASH MANAGEMENT FUND INC, DREYFUS STOCK INDEX FUND INC, Lazard World Dividend Income Fund Inc, DIVIDEND Fund Inc by Gabelli]","[Stock Dividend Fund Inc, FMI COMMON STOCK FUND INC, GENERAL MONEY MARKET FUND INC, NATIONAL INVESTORS CASH MANAGEMENT FUND INC, DREYFUS STOCK INDEX FUND INC, Lazard World Dividend Income Fund Inc, DIVIDEND Fund Inc by Gabelli]",[],"[0.0000, 0.0805, 0.0990, 0.1126, 0.1134, 0.1138, 0.1161]"


In [9]:
text = 'MMEX Resources Corporation'

%time get_codes (lp, text, vocab='normalized_name')

CPU times: user 20.3 ms, sys: 4.55 ms, total: 24.8 ms
Wall time: 927 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,MMEX Resources Corporation,0,25,MMEX Resources Corp,"[MMEX Resources Corp, ANTERO RESOURCES Corp, Medgold Resources Corp, CNX Resources Corp, Evrim Resources Corp, Centaurus Resources Corp, Kopr Resources Corp, Arcis Resources Corp, Greenpro Resources Corp, Rockdale Resources Corp, Oxbow Resources Corp, Kaeland Resources Corp, Oneida Resources Corp, Cheval Resources Corp, Nogales Resources Corp, Timberline Resources Corp, Richland Resources Corp, Element Resources Corp, Antero Resources Finance Corp, Resource Capital Corp, Kinetic Resources Corp, Biopower Operations Corp, International Frontier Resources Corp, Targa Resources Corp, Petrus Resources Corp, TRIWEST MANAGMEENT RESOURCES CORP, MALLON RESOURCES CORP]","[MMEX Resources Corp, ANTERO RESOURCES Corp, Medgold Resources Corp, CNX Resources Corp, Evrim Resources Corp, Centaurus Resources Corp, Kopr Resources Corp, Arcis Resources Corp, Greenpro Resources Corp, Rockdale Resources Corp, Oxbow Resources Corp, Kaeland Resources Corp, Oneida Resources Corp, Cheval Resources Corp, Nogales Resources Corp, Timberline Resources Corp, Richland Resources Corp, Element Resources Corp, Antero Resources Finance Corp, Resource Capital Corp, Kinetic Resources Corp, Biopower Operations Corp, International Frontier Resources Corp, Targa Resources Corp, Petrus Resources Corp, TRIWEST MANAGMEENT RESOURCES CORP, MALLON RESOURCES CORP]",[],"[0.0000, 0.0580, 0.0626, 0.0674, 0.0679, 0.0699, 0.0705, 0.0787, 0.0886, 0.0971, 0.1048, 0.1099, 0.1111, 0.1113, 0.1133, 0.1241, 0.1301, 0.1333, 0.1375, 0.1384, 0.1467, 0.1469, 0.1469, 0.1496, 0.1521, 0.1546, 0.1563]"


In [10]:
text = 'BAKER HUGHES'

%time get_codes (lp, text, vocab='normalized_name')

CPU times: user 18.8 ms, sys: 2.42 ms, total: 21.2 ms
Wall time: 788 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,BAKER HUGHES,0,11,BAKER HUGHES INC,"[BAKER HUGHES INC, BAKER DARRYL S, Payne Fletcher, BAKER FELIX, Baker Hughes Co, Baker Malcolm, BAKER HAROLD, Baker Douglas D]","[BAKER HUGHES INC, BAKER DARRYL S, Payne Fletcher, BAKER FELIX, Baker Hughes Co, Baker Malcolm, BAKER HAROLD, Baker Douglas D]",[],"[0.1144, 0.1657, 0.1782, 0.1806, 0.1864, 0.1876, 0.1919, 0.1985]"


### Find Company IRS Number

An employer identification number (EIN) is a nine-digit number assigned by the `IRS`. It's used to identify the tax accounts of employers and certain others who have no employees. The IRS uses the number to identify taxpayers who are required to file various business tax returns. EINs are used by employers, sole proprietors, corporations, partnerships, non-profit associations, trusts, estates of decedents, government agencies, certain individuals, and other business entities.

![image.png](attachment:01d5797e-94ae-4d7a-acb6-52cfa88ca194.png)


In [11]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("ner_chunk") \
      .setOutputCol("sentence_embeddings")
    
resolver = finance.SentenceEntityResolverModel.pretrained("finel_edgar_irs", "en", "finance/models") \
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("irs_code")\
      .setDistanceFunction("EUCLIDEAN")

pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          embeddings,
          resolver])

lp = LightPipeline(pipelineModel)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
finel_edgar_irs download started this may take some time.
[OK!]


In [12]:
text = 'SPO Global INC.'

%time get_codes (lp, text, vocab='irs_code')

CPU times: user 29.9 ms, sys: 4.74 ms, total: 34.7 ms
Wall time: 785 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,SPO Global INC.,0,14,251411971,"[251411971, 262801338, 471208256, 260338889, 0, 870645794, 953797580]","[SPO Global Inc, JPX Global Inc, EliteSoft Global Inc, GLOBAL NUTECH INC, PRGX GLOBAL INC, Global Inc, MIKOJO Inc]",[],"[0.0000, 0.2017, 0.2046, 0.2047, 0.2126, 0.2142, 0.2150]"


In [13]:
text = 'WOODLEY FARRA MANION PORTFOLIO MANAGEMENT Incorporated'

%time get_codes (lp, text, vocab='irs_code')

CPU times: user 14.3 ms, sys: 2.35 ms, total: 16.6 ms
Wall time: 756 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,WOODLEY FARRA MANION PORTFOLIO MANAGEMENT Incorporated,0,53,351964649,"[351964649, 0, 472447468, 463303127, 200467860, 863425507, 264766975, 263267232]","[WOODLEY FARRA MANION PORTFOLIO MANAGEMENT INC, Altegris Portfolio Management Inc, Connective Portfolio Management LLC, Comprehensive Portfolio Management LLC, Minneapolis Portfolio Management Group LLC, ReAlpha Asset Management Inc, Carnegie Capital Asset Management LLC, SPRING CAPITAL MANAGEMENT LLC]",[],"[0.0000, 0.1259, 0.1309, 0.1530, 0.1627, 0.1636, 0.1638, 0.1658]"


In [14]:
text = 'Alzamend Neuro INC'

%time get_codes (lp, text, vocab='irs_code')

CPU times: user 22.4 ms, sys: 3.78 ms, total: 26.2 ms
Wall time: 780 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,Alzamend Neuro INC,0,17,811822909,"[811822909, 0, 262908274, 352528215]","[Alzamend Neuro Inc, Neuro Kinetics Inc, Ra Pharmaceuticals Inc, Cocrystal Pharma Inc]",[],"[0.0000, 0.1156, 0.1464, 0.1466]"


## Sentence Entity Resolver (CRUNCHBASE)
[Crunchbase Homepage](https://www.crunchbase.com/)

![image.png](attachment:eb4df463-fe67-48d7-a8cc-83a993234fc7.png)

- Crunchbase is a platform for gaining awareness about business information about private and public companies. Originally built to track startups, the Crunchbase website contains information on public and private companies on a global scale.

Here we will normalize company names with the Crunchbase Database

In [15]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("ner_chunk") \
      .setOutputCol("sentence_embeddings")
    
resolver = finance.SentenceEntityResolverModel.pretrained("finel_crunchbase_companynames", "en", "finance/models")\
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("name")\
      .setDistanceFunction("EUCLIDEAN")

pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          embeddings,
          resolver])

lp = LightPipeline(pipelineModel)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
finel_crunchbase_companynames download started this may take some time.
[OK!]


In [16]:
text = 'Shwrm'

%time get_codes (lp, text, vocab='name')

CPU times: user 13.5 ms, sys: 5.46 ms, total: 18.9 ms
Wall time: 225 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,Shwrm,0,4,Shwrüm,"[Shwrüm, Xervmon Inc, ADVANCED CREDIT TECHNOLOGIES, Quickcomm Software Solutions, citysocializer, SurgiQuest, ShoutNow, Reset Therapeutics, MoneyReef, TopiVert, Brevity, Phoenix Health and Safety, Learnpedia Edutech Solutions, Stumpedia]","[Shwrüm, Xervmon Inc, ADVANCED CREDIT TECHNOLOGIES, Quickcomm Software Solutions, citysocializer, SurgiQuest, ShoutNow, Reset Therapeutics, MoneyReef, TopiVert, Brevity, Phoenix Health and Safety, Learnpedia Edutech Solutions, Stumpedia]",[],"[0.0000, 0.0436, 0.0448, 0.0471, 0.0488, 0.0488, 0.0491, 0.0497, 0.0497, 0.0500, 0.0500, 0.0504, 0.0507, 0.0507]"


In [17]:
text = 'Waywire'

%time get_codes (lp, text, vocab='name')

CPU times: user 15 ms, sys: 2.58 ms, total: 17.6 ms
Wall time: 223 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,Waywire,0,6,#waywire,"[#waywire, Limonetik, Totally Interactive Weather, ThoughtFocus, 2345.com, WebNotes, Synovex, relocality, Grab Media]","[#waywire, Limonetik, Totally Interactive Weather, ThoughtFocus, 2345.com, WebNotes, Synovex, relocality, Grab Media]",[],"[0.0000, 0.0431, 0.0434, 0.0441, 0.0443, 0.0445, 0.0452, 0.0458, 0.0459]"


In [18]:
text = 'alist games'

%time get_codes (lp, text, vocab='name')

CPU times: user 12.5 ms, sys: 761 µs, total: 13.3 ms
Wall time: 231 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,alist games,0,10,[a]list games,"[[a]list games, Headright Games, Komar Games, Matchmove Games, Kreeda Games, Hardscore Games, TinyMob Games]","[[a]list games, Headright Games, Komar Games, Matchmove Games, Kreeda Games, Hardscore Games, TinyMob Games]",[],"[0.0000, 0.1525, 0.1629, 0.1843, 0.1901, 0.1919, 0.1931]"


## Sentence Entity Resolver (NASDAQ)
[Nasdaq Homepage](https://www.nasdaq.com/)

![image.png](attachment:ac942854-4b4b-4c68-9c24-4899bf0f8377.png)

- `The Nasdaq Stock Market` (National Association of Securities Dealers Automated Quotations Stock Market) is an American stock exchange based in New York City. It is ranked second on the list of stock exchanges by market capitalization of shares traded, behind the New York Stock Exchange

### Find Ticker Symbol

`Ticker Symbol` is the use of letters to represent shares that are traded on the stock market, and it is mainly a combination of two , three or four alphabets that is unique and easy for investors to identify and buy/sell that particular stock with the help of this symbol on the stock exchange. Symbols with four or more letters generally denote securities traded on the American stock exchange and NASDAQ.


Here we will find the ticker symbol of each company on Nasdaq

In [19]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

embeddings = nlp.UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
      .setInputCols("ner_chunk") \
      .setOutputCol("sentence_embeddings")
    
resolver = finance.SentenceEntityResolverModel.pretrained("finel_names2tickers", "en", "finance/models") \
      .setInputCols(["ner_chunk", "sentence_embeddings"]) \
      .setOutputCol("name")\
      .setDistanceFunction("EUCLIDEAN")

pipelineModel = PipelineModel(
      stages = [
          documentAssembler,
          embeddings,
          resolver])

lp = LightPipeline(pipelineModel)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
finel_names2tickers download started this may take some time.
[OK!]


In [20]:
text = 'Alcoa'

%time get_codes (lp, text, vocab='name')

CPU times: user 10.8 ms, sys: 3.18 ms, total: 14 ms
Wall time: 75.9 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,Alcoa,0,4,AA,"[AA, GLW, CVX, HUBB, IMV, ENB, LDL, SRT, MOSY, OMCL, RDNT, IBKR, CVCO, EVI, EYEN, PFPT, TMST]","[AA, GLW, CVX, HUBB, IMV, ENB, LDL, SRT, MOSY, OMCL, RDNT, IBKR, CVCO, EVI, EYEN, PFPT, TMST]",[],"[0.0000, 0.1926, 0.1980, 0.2066, 0.2099, 0.2144, 0.2192, 0.2200, 0.2242, 0.2242, 0.2261, 0.2272, 0.2281, 0.2285, 0.2294, 0.2297, 0.2310]"


In [21]:
text = 'Azurrx biopharma'

%time get_codes (lp, text, vocab='name')

CPU times: user 13.1 ms, sys: 4.71 ms, total: 17.8 ms
Wall time: 76.7 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,Azurrx biopharma,0,15,AZRX,"[AZRX, SHSP, ULBI, CATS, ELMD, MTNB, VSH, ADMA, CYTR, FCFS, AXR, TBPH, CCXI, KMPH, CBTX, EXLS, MAXR, HEB, NTRP, PRPO, MYO, IDA, TORC, VCEL, SVMK]","[AZRX, SHSP, ULBI, CATS, ELMD, MTNB, VSH, ADMA, CYTR, FCFS, AXR, TBPH, CCXI, KMPH, CBTX, EXLS, MAXR, HEB, NTRP, PRPO, MYO, IDA, TORC, VCEL, SVMK]",[],"[0.0000, 0.1319, 0.1439, 0.1464, 0.1474, 0.1497, 0.1506, 0.1521, 0.1525, 0.1540, 0.1548, 0.1548, 0.1549, 0.1556, 0.1559, 0.1565, 0.1568, 0.1576, 0.1591, 0.1594, 0.1601, 0.1603, 0.1606, 0.1611, 0.1612]"


In [22]:
text = 'apple'

%time get_codes (lp, text, vocab='name')

CPU times: user 12.4 ms, sys: 2.08 ms, total: 14.4 ms
Wall time: 75.4 ms


,chunks,begin,end,code,all_codes,resolutions,all_k_aux_labels,all_distances
0,apple,0,4,AAPL,"[AAPL, XRX, MSFT, PEP, APLE, HPQ, NOW, ZAGG, MOV]","[AAPL, XRX, MSFT, PEP, APLE, HPQ, NOW, ZAGG, MOV]",[],"[0.0000, 0.3793, 0.3925, 0.4013, 0.4087, 0.4287, 0.4562, 0.4620, 0.4634]"
